In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 100
INITIAL_LEARNING_RATE = 0.001

In [4]:
# Define the data generators with validation split
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/StormVision',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Use 'categorical' for multi-label classification
    subset='training')

validation_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/StormVision',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

# Split validation data into validation and test sets
test_generator = data_gen.flow_from_directory(
    '/content/drive/MyDrive/Final_Year_Weather_Classification/Dataset/StormVision',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False)


Found 2582 images belonging to 4 classes.
Found 643 images belonging to 4 classes.
Found 643 images belonging to 4 classes.


In [5]:
# Function to create model
def create_model(num_classes):
    input_tensor = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='sigmoid')(x)  # Sigmoid activation for multi-label classification

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [6]:
# Function to compile and train model
def compile_and_train_model(model, train_generator, validation_generator, num_classes):
    model.compile(optimizer=Adam(learning_rate=INITIAL_LEARNING_RATE),
                  loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
                  metrics=['accuracy'])
    history = model.fit(train_generator,
                        epochs=EPOCHS,
                        validation_data=validation_generator)
    return history

In [7]:
# Function to evaluate model
def evaluate_model(model, test_generator):
    test_generator.reset()
    predictions = model.predict(test_generator)
    y_pred = (predictions > 0.5).astype("int32")  # Thresholding predictions at 0.5
    y_true = test_generator.classes
    print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))
    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

In [8]:


# Function to plot training history
def plot_history(history, model_name):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Create and train models
# Adjust the number of classes based on your dataset
num_classes = len(train_generator.class_indices)
nightnet = create_model(num_classes=num_classes)
history_nightnet = compile_and_train_model(nightnet, train_generator, validation_generator, num_classes=num_classes)


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
 5/81 [>.............................] - ETA: 29:47 - loss: 2.5567 - accuracy: 0.6438

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-324151f3ab24>", line 29, in <cell line: 29>
    history_nightnet = compile_and_train_model(nightnet, train_generator, validation_generator, num_classes=num_classes)
  File "<ipython-input-6-d662bf5f6544>", line 6, in compile_and_train_model
    history = model.fit(train_generator,
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/pytho

TypeError: object of type 'NoneType' has no len()

In [ ]:
loaded_model = load_model("/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/CNN_stormvision.h5")
print("Model loaded successfully.")

In [ ]:
nightnet.load_weights("/content/drive/MyDrive/Final_Year_Weather_Classification/CNN_Resuidal_Code/CNN_StromVision100ep.h5")
print("Model weights loaded successfully.")
